In [ ]:
import numpy as np
import pandas as pd
import os
from datetime import timedelta
import matplotlib.pyplot as plt

from grid_and_bathy import get_grid
from configs.config_object import ConfigObject

  "current_project": {
    "lake_name": "lucerne",
    "grid_config_name": "100m",
    "start_date": "20250701",
    "end_date": "20250825",
    "reference_date": "20250701",
    "weather_model": "icon",
    "computer_config": "kball_laken",
    "template_folder": ["default","exch2","postprocessing"],
    "initialization_type": "simstrat"
  },

In [ ]:
config = ConfigObject('../config.json')

In [ ]:
time_resolution = timedelta(hours=1)

In [ ]:
path_grid = config.paths.grid_folder_path
output_folder = '../99-output/binary_data/'
grid_resolution = config.grid_resolution
start_date=config.start_date
end_date=config.end_date
bathy_path = config.paths.bathy_path
rotation_model = config.rotation

In [ ]:
mitgcm_grid = get_grid(path_grid)

# Get binary files characteristics

In [ ]:
nx = len(mitgcm_grid.x)
ny = len(mitgcm_grid.y)
nt = (pd.to_datetime(end_date, format="%Y%m%d") - pd.to_datetime(start_date, format="%Y%m%d")) / time_resolution +1

In [ ]:
date_list = pd.date_range(start=pd.to_datetime(start_date, format="%Y%m%d"), periods=int(nt), freq=time_resolution)

In [ ]:
base_arr = np.zeros((int(nt),int(ny),int(nx)))

# Station's measurements

In [ ]:
sta_folder_path = r'C:\Users\leroquan\Documents\Data\meteo\stations meteo\lucerne'
sta_codes = ['alt','ges','luz']
sta_coords = [(46.887069,8.621894),(46.996069,8.523442),(47.036439,8.301022)]

In [ ]:
def get_sta_df(sta_folder_path, sta_code, date_list, rotation_model):
    df_sta = pd.read_csv(os.path.join(sta_folder_path, f'ogd-smn_{sta_code}_h_recent.csv'), sep=';')
    df_sta['reference_timestamp'] = pd.to_datetime(
        df_sta['reference_timestamp'], format="%d.%m.%Y %H:%M"
        )
    df_sta['time'] = df_sta['reference_timestamp']
    df_sta = df_sta.set_index('reference_timestamp')
    df_interp = df_sta[['fu3010h0','dkl010h0']].reindex(date_list).interpolate(method="linear")
    df_interp['corrected_direction_in_degrees'] = df_interp['dkl010h0'] + rotation_model
    df_interp['u'] = df_sta['fu3010h0'] / 3.6 * np.sin(np.radians(df_interp['corrected_direction_in_degrees']))
    df_interp['v'] = df_sta['fu3010h0'] / 3.6 * np.cos(np.radians(df_interp['corrected_direction_in_degrees']))

    return df_interp # Hourly averaged speed and direction of the wind (0°=N)

In [ ]:
dfs_sta = {}
for sta_code in sta_codes:
    dfs_sta[sta_code] = get_sta_df(sta_folder_path, sta_code, date_list, rotation_model)

In [ ]:
dfs_sta[sta_codes[0]][['u','v']].plot()

# Import bathy

In [ ]:
with open(bathy_path, 'rb') as fid:
    binary_bathy = np.fromfile(fid, dtype='>f8')

In [ ]:
bathy_arr = np.reshape(binary_bathy, (int(ny),int(nx)))

In [ ]:
plt.imshow(bathy_arr)
plt.gca().invert_yaxis()

# Define extrapolation zones

In [ ]:
sta_codes = ['alt','ges','luz']
sta_zones = {'luz':[[(0,100),(0,ny)],
                    [(100,120),(85,130)],
                    [(120,145),(91,130)],
                    [(145,150),(95,130)]],
             'ges':[[(100,230),(0,ny)]],
             'alt':[[(230,nx),(0,ny)]]}

# Apply measured speed to the zones

In [ ]:
u_arr = np.zeros((int(nt),int(ny),int(nx)))
v_arr = np.zeros((int(nt),int(ny),int(nx)))
for sta_code in sta_codes:
    u_sta = dfs_sta[sta_code]['u'].values
    v_sta = dfs_sta[sta_code]['v'].values
    for zones in sta_zones[sta_code]:
        u_arr[:,zones[1][0]:zones[1][1],zones[0][0]:zones[0][1]] = u_sta[:, np.newaxis, np.newaxis]
        v_arr[:,zones[1][0]:zones[1][1],zones[0][0]:zones[0][1]] = v_sta[:, np.newaxis, np.newaxis]

In [ ]:
plt.figure(figsize=(10,15))
plt.imshow(bathy_arr)
plt.imshow(u_arr[9], alpha=0.5)
plt.gca().invert_yaxis()

In [ ]:
plt.quiver(dfs_sta['alt']['u'][9], dfs_sta['alt']['v'][9], color='r', scale=5)
plt.quiver(0.01, 0.01, scale=0.1)

# Write velocity files

In [ ]:
def write_array_to_binary(path_fname, data):
    dtype = '>f8'  # big-endian and precision 64 (small-endian with precision 32 is '<f4')
    data = data.astype(dtype)

    # Write to file
    fid = open(path_fname, 'wb')
    data.tofile(fid)
    fid.close()

In [ ]:
write_array_to_binary(os.path.join(output_folder, 'u10.bin'), u_arr)
write_array_to_binary(os.path.join(output_folder, 'v10.bin'), v_arr)